In [1]:
import os
import shutil
import glob
import json
import random
import scipy.io as sio
import pandas as pd
import numpy as np

In [2]:
raw_data_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_rawfmri'

bids_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_bidsfmri'

subnums = ['601', '609', '611', '619', '621', '629']

sessions = ['01', '02', '03']

data_types = ['anat', 'func']

misc_files = ['dataset_description.json', 'participants.json', 'README', 'CHANGES']

In [3]:
timing_mats = glob.glob(raw_data_path + '/*/*/*.mat')

In [17]:
cur_timing = timing_mats[10]
cur_timing

'/Users/zeynepenkavi/Downloads/overtrained_decisions_rawfmri/sub-601/ses-02/taskYN_subj601_day7_session3.mat'

In [18]:
tmp = sio.loadmat(cur_timing, squeeze_me=True)

In [19]:
timing = tmp['timing']
timing_vals = timing.item()

In [20]:
if len(timing.dtype)>6:
    timing_keys = ['Begin', 'onset', 'feedbackOn', 'crossStart', 'crossEnd', 'End', 'session'] #session only exists for the later YN run
    tmp_dict = dict(zip(timing_keys, timing_vals))
    cut_dict = dict()
    for k in timing_keys:
        half_len = int(len(tmp_dict[k])/2)
        cut_dict[k] = tmp_dict[k][-half_len:]
else:
    timing_keys = ['Begin', 'onset', 'feedbackOn', 'crossStart', 'crossEnd', 'End']
    cut_dict = dict(zip(timing_keys, timing_vals))

run_begin = float(cut_dict['Begin'])
run_end = float(cut_dict['End']) - run_begin
del cut_dict['Begin']
del cut_dict['End']
if 'session' in timing_keys:
    del cut_dict['session']

timing_df = pd.DataFrame(cut_dict) - run_begin
stim_timing = pd.DataFrame({'onset': timing_df['onset'], 'duration':  timing_df['feedbackOn'] - timing_df['onset'], 'trial_type': 'stim'})
feedback_timing = pd.DataFrame({'onset': timing_df['feedbackOn'], 'duration':  np.concatenate([np.array(timing_df['crossStart'][1:]), [run_end]]) - timing_df['feedbackOn'], 'trial_type': 'feedback'})
cross_timing = pd.DataFrame({'onset': timing_df['crossStart'][1:], 'duration': timing_df['crossEnd'][1:] - timing_df['crossStart'][1:], 'trial_type': 'fixCross'})
run_events = pd.concat([stim_timing, feedback_timing, cross_timing]).sort_values(by=['onset']).reset_index(drop=True)

In [21]:
run_events

,onset,duration,trial_type
0,10.010370,1.798840,stim
1,11.809210,1.031124,feedback
2,12.840334,3.574478,fixCross
3,16.414850,1.597711,stim
4,18.012561,1.033248,feedback
...,...,...,...
594,778.188129,1.479478,stim
595,779.667606,1.032012,feedback
596,780.699619,2.412931,fixCross
597,783.112559,1.236549,stim
